
Dieses Repository enthält
- Daten
    - Daten des Statistischen Bundesamtes zur Sterbestatistik im Corona-Jahr 2020 und den Vorjahren,
    - Populationsdaten nach Geschlecht und Alter
- Erschließen der Daten-Dateien mit julia-Funktionen
- Auswertung des durchschnittlichen kausalen Jahres-Effekts, adjustiert für Alter und Geschlecht.

Ziele: 
- Sterbestatistik angleichen bzgl. Alter und Geschlecht, um verfälschende Effekte einer alternden Gesellschaft   zu korrigieren.
    Die Auswertung macht die Todeszallzahlen 
    anhand von Alter und Geschlecht 
    mit den sich ändernden Verhältnissen einer alternden Gesellschaft 
    statistisch zwischen den Jahren vergleichbar.

- Exploration öffentliche Daten: 
   - Wie leicht können veröffentlichte Daten für reproduzierbare Forschung erschlossen werden? **nicht leicht**
   - Veröffentlichung eines Skripts zur Datenerschließung. 
   - Wie gut eignet sich julia als Sprache, um Datenerschließung dezentral zu formalisieren und zu organisieren?




# Datengrundlage
## Informations-Mycel
Die Daten des statistischen Bundesamtes sind Daten verfügbar auf die eine oder andere Weise, d.h. 
- nach Login
- Auswahl von Spalten
Es existiert ein API zur Abfrage der Daten des statistischen Bundesamtes.
Dieses wurde hier nicht verwendet, da die Definition der Schnittstelle sehr umfangreich ist.
Für spezifische Daten wie hier ist das manuelle Herunterladen und Auslesen mit Hilfsfunktionen schneller umzusetzen.


In [1]:
cd("/home/gregor/dev/julia/OrGitBot/repositories/Sterbestatistik")
using Pkg
Pkg.activate(".")

 Activating environment at `~/dev/julia/OrGitBot/repositories/Sterbestatistik/Project.toml`


In [365]:
jahre = 2016:2020
geschlechter = ["Männlich", "Weiblich"] # , "Insgesamt"
MAX=200
altersgruppen = [ 0:30, [a:a+5 for a in 30:5:80]..., 85:MAX ];

In [426]:
function deaths(; alter::UnitRange, geschlecht, jahr, kw=missing)
	if kw === missing
		return sum(Int[ deaths(;alter=alter,geschlecht=geschlecht, jahr=jahr,kw=i)
             for i in 1:53
             if deaths(;alter=alter,geschlecht=geschlecht, jahr=jahr, kw=i) isa Integer])
			 
	end
    @assert geschlecht in geschlechter
    if alter==85:MAX
        return deaths(;alter=85:90,geschlecht=geschlecht, jahr=jahr, kw=kw)+
            deaths(;alter=90:95,geschlecht=geschlecht, jahr=jahr, kw=kw)+
            deaths(;alter=95:MAX,geschlecht=geschlecht, jahr=jahr, kw=kw)
    end
    # @assert alter isa UnitRange && alter.stop-alter.start==5 && alter.start<=95
    if alter.start >= 95
        alter = 95:MAX
        error()
    end    
    if alter.stop <= 30
        alter = 0:30
        error()
    end
    # skipping 11 lines,
    zeile = 12 +
        # 16 lines blocks for years,
        (2020-jahr)*16 + 
        # 5 year steps beginning age 30
        max(0, (alter.start-30)/5)
    ## look up gender tab
    tab = d["D_2016_2020_KW_AG_$geschlecht"]
    # and check in cw column, skipping first 3
    tab[convert(Int,zeile), kw+3]
end
deaths(alter=40:45,jahr=2020, geschlecht="Männlich")

3917

## **Datensatz** Sterbestatistik
Ziel: julia Funktion Sterbefälle (absolut) spezifisch für Alter, Geschlecht, Jahr, Kalenderwoche

Daten Quelle: [statistischen Bundesamtes](https://www.destatis.de/DE/Themen/Gesellschaft-Umwelt/Bevoelkerung/Sterbefaelle-Lebenserwartung/Tabellen/sonderauswertung-sterbefaelle.html) (Download [xlsx](https://www.destatis.de/DE/Themen/Gesellschaft-Umwelt/Bevoelkerung/Sterbefaelle-Lebenserwartung/Tabellen/sonderauswertung-sterbefaelle.xlsx?__blob=publicationFile)),
Verwendung des Tabellenblatt mit Aufschlüsselung nach Kalenderwoche.

CAVEAT:
- **Altersgruppen 0-30, dann in 5-Jahres-Schritten bis 95**

In [254]:
using XLSX
d = XLSX.readxlsx("data/sonderauswertung-sterbefaelle.xlsx")
"""
Nachschlagefunktion 
"""
function S(; alter, geschlecht="", jahr, kw)
    @assert geschlecht in geschlechter
    geschlecht == "Insgesamt" && (geschlecht="Ins")
    # @assert alter isa UnitRange && alter.stop-alter.start==5 && alter.start<=95
    if alter.start >= 95
        alter = 95:MAX
    end    
    if alter.stop <= 30
        alter = 0:30
    end
    zeile = 11 + (2020-jahr)*16 + max(0, (alter.start-25)/5)
    # alter => 
    d["D_2016_2020_KW_AG_$geschlecht"][convert(Int,zeile),kw+3]
end
# todo: merge into 1 function 
S_(;alter,kw...) = 
    if alter==85:MAX
        S(;alter=85:90,kw...)+S(;alter=90:95,kw...)+S(;alter=95:MAX,kw...)
    else
        S(;alter=alter,kw...)
    end
"""
Aggregationsfunktion
"""
Sterbefälle(;kw...) =  sum(Int[ S(;kw...,kw=i) for i in 1:53 if S(;kw...,kw=i) isa Integer])
                
# Example for testing:                
# [ a => Sterbefälle(; alter=a, geschlecht="Weiblich", jahr=2020) 
#   for a in altersgruppen ]


## Bevölkerungsstatistik (Pyramide)
Ziel: Bevölkerungspyramide (absolut): N | Alter, Geschlecht, Jahr

Daten-Quelle:
[Statistisches Bundesamt, Tabelle 12411-0006](https://www-genesis.destatis.de/genesis//online?operation=table&code=12411-0006&bypass=true&levelindex=0&levelid=1612115589154#abreadcrumb)

CAVEAT:
- **Alter <0, in 1-Jahres-Schritte bis 85, und aggregiert darüber**


[Podcast](https://www.destatis.de/DE/Themen/Gesellschaft-Umwelt/Bevoelkerung/Sterbefaelle-Lebenserwartung/Podcast/podcast-sterbefaelle.html)


Alternative Daten (nicht verwendet):
- [Pyramide](https://service.destatis.de/bevoelkerungspyramide/index.html#!y=2018&v=2)

In [333]:
using CSV, DataFrames, StringEncodings

pyramide = CSV.File(
    open(read, "data/12411-0006.csv", enc"ISO-8859-1");
    skipto=9, footerskip=4,
    delim=';', decimal=',',
    dateformat="dd.mm.yyy", 
    header=["Stichtag", "Altersgruppe","Männlich","Weiblich","Insgesamt" ]) |> DataFrame
pyramide = pyramide[pyramide.Altersgruppe .!= "Insgesamt",:]

using CombinedParsers

jahrp = Either("unter 1 Jahr" => 0:1,
               map(a -> a:a, 
                 Sequence(1, CombinedParsers.Numeric(Int),"-Jährige")),
               "85 Jahre und mehr" => 85:MAX-1,
               "Insgesamt" => 0:MAX-1)

pyramide[!,"Altersgruppe"] = [ parse(jahrp, v) for v in pyramide[!,2] ];

In [325]:
using Dates
iseq(x,y) = [ e.start >= y.start && e.stop < y.stop for e in x ]
function Bevölkerung(; jahr, geschlecht="Insgesamt", alter)
    sum(pyramide[(pyramide[:,1] .== Date(jahr-1,12,31)) .& iseq(pyramide[:,2], alter), geschlecht])
end

Bevölkerung (generic function with 1 method)

## Tabellarische Zusammenstellung
Anhand der Nachschlagefunktionen wird ein DataFrame Tabellenblatt erstellt.

In [367]:
# sterberaten daten
srd = [ 
  ( alter = a,
    geschlecht=g, 
    jahr=j,
    N=Bevölkerung(alter=a, geschlecht=g, jahr=j),
    S=Sterbefälle(alter=a, geschlecht=g, jahr=j))
  for a in altersgruppen
  for g in geschlechter
  for j in jahre ] |> DataFrame;

srd

,alter,geschlecht,jahr,N,S
,UnitRan…,String,Int64,Int64,Int64
1,0:30,Männlich,2016,12980804,5207
2,0:30,Männlich,2017,13049733,4827
3,0:30,Männlich,2018,13035105,4982
4,0:30,Männlich,2019,12994349,4820
5,0:30,Männlich,2020,12943028,4620
6,0:30,Weiblich,2016,12065845,2909
7,0:30,Weiblich,2017,12112491,2877
8,0:30,Weiblich,2018,12117815,2814
9,0:30,Weiblich,2019,12100550,2688


In [342]:
gsrd = groupby(srd, :jahr)
N_sum = Dict(r[1] => r[2] for r in eachrow(combine(gsrd, :N=>sum)))

Dict{Int64,Int64} with 5 entries:
  2017 => 82521653
  2019 => 83019213
  2018 => 82792351
  2020 => 83166711
  2016 => 82175684

In [370]:
# Test whether srd groups sum to full population
# pyramide[pyramide.Stichtag.==Date(2015,12,31),:][40:end,:]
altersgruppen
Bevölkerung(jahr=2016, geschlecht="Männlich", alter=0:MAX)
[ a => Bevölkerung(alter=a, geschlecht="Weiblich", jahr=2015) for a in altersgruppen ]
Bevölkerung(alter=altersgruppen[2], geschlecht="Männlich", jahr=2020),
S(alter=altersgruppen[1], geschlecht="Männlich", jahr=2020, kw=1),
Sterbefälle(alter=altersgruppen[2], geschlecht="Männlich", jahr=2020)

# Test der Bevölkerungstabelle-Summe: erwarte Gesamtbevölkerung zum jeweiligen Stichtag
gsrd = groupby(pyramide, :Stichtag)
tmp = combine(gsrd, :Insgesamt=>sum)
tmp[:,:df] = [ get(N_sum, year(d)+1, missing) for d in tmp[:,1] ]
tmp

,Stichtag,Insgesamt_sum,df
,Date,Int64,Int64?
1,2014-12-31,81197537,missing
2,2015-12-31,82175684,82175684
3,2016-12-31,82521653,82521653
4,2017-12-31,82792351,82792351
5,2018-12-31,83019213,83019213
6,2019-12-31,83166711,83166711


### Sterbewkeiten=Erwartungswerte | Alter, Geschlecht, Jahr
Photo Rolf (1): 
$$
P^{2016}( + | sex, alter)
$$
Gregor
$$
\frac{S}{N} | Alter, Geschlecht, Jahr
$$

Sterblichkeit adjustiert auf Referenzjahr 2020 (Samuel Eckert?)
s(Jahr=x) = E[ (S/N | Alter, Geschlecht, Jahr=x) * (N | Alter, Geschlecht, Jahr=2020) ]


s(Jahr=x) = E[ (S/N | Alter, Geschlecht, Jahr=x) * E(N | Alter, Geschlecht) ]



Bevölkerungspyramide (absolut): N | Alter, Geschlecht, Jahr
Sterbetabellen (absolut): S | Alter, Geschlecht, Jahr



In [372]:
# Mittlere Wahrscheinlichkeit
srd[:,:P] = srd.N ./ [ N_sum[j] for j in srd.jahr ]
srd[:,:cell] = collect(zip(srd.alter,srd.geschlecht))
gsrd = groupby(srd, :cell)
P_mean = Dict([ r[1] => r[2] 
        for r in eachrow(combine(gsrd, :P=>mean))])

# todo: plot?

Dict{Tuple{UnitRange{Int64},String},Float64} with 26 entries:
  (50:55, "Männlich")  => 0.0420866
  (55:60, "Männlich")  => 0.0386273
  (40:45, "Männlich")  => 0.0296582
  (75:80, "Männlich")  => 0.0223913
  (80:85, "Weiblich")  => 0.0206547
  (85:200, "Männlich") => 0.00873846
  (65:70, "Männlich")  => 0.026885
  (45:50, "Männlich")  => 0.0360932
  (30:35, "Männlich")  => 0.0330045
  (40:45, "Weiblich")  => 0.0292099
  (55:60, "Weiblich")  => 0.0387137
  (80:85, "Männlich")  => 0.0144088
  (60:65, "Weiblich")  => 0.0333426
  (60:65, "Männlich")  => 0.0319194
  (35:40, "Männlich")  => 0.031457
  (70:75, "Weiblich")  => 0.0238699
  (65:70, "Weiblich")  => 0.0293687
  (75:80, "Weiblich")  => 0.0279475
  (45:50, "Weiblich")  => 0.0354478
  (85:200, "Weiblich") => 0.0187726
  (35:40, "Weiblich")  => 0.0306629
  (30:35, "Weiblich")  => 0.0312535
  (0:30, "Weiblich")   => 0.146173
  (50:55, "Weiblich")  => 0.0413013
  (0:30, "Männlich")   => 0.157139
  (70:75, "Männlich")  => 0.0208736

In [415]:
adjusted = [
    ( jahr=j, alter=a, geschlecht=g,
      kw=kw,
    N = Bevölkerung(jahr=j,alter=a,geschlecht=g),
    S = S_(jahr=j, alter=a, geschlecht=g, kw=kw),
    marginalP = P_mean[ (a,g) ])
    for a in altersgruppen, 
        kw in 1:53, 
        g in geschlechter,
        j in jahre
    if S(jahr=j,alter=a,geschlecht=g,kw=kw) isa Int] |> DataFrame
            
# Formel 3
adjusted[:,:Sadj2020] = ( adjusted.S ./ adjusted.N ) .* adjusted.marginalP * N_sum[2020] 
adjusted[:,:Sadj] = ( adjusted.S ./ adjusted.N ) .* adjusted.marginalP .* [ N_sum[j] for j in adjusted.jahr ]

adjusted[1:2,:]

,jahr,alter,geschlecht,kw,N,S,marginalP,Sadj2020
,Int64,UnitRan…,String,Int64,Int64,Int64,Float64,Float64
1,2016,0:30,Männlich,1,12980804,95,0.157139,95.6434
2,2016,30:35,Männlich,1,2647069,37,0.0330045,38.3671


In [410]:
adjusted[(adjusted.jahr.==2020) .& (adjusted.kw.==1),:]

,jahr,alter,geschlecht,kw,N,S,marginalP,Sadj
,Int64,UnitRan…,String,Int64,Int64,Int64,Float64,Float64
1,2020,0:30,Männlich,1,12943028,84,0.157139,84.8157
2,2020,30:35,Männlich,1,2830265,54,0.0330045,52.3708
3,2020,35:40,Männlich,1,2683121,54,0.031457,52.6527
4,2020,40:45,Männlich,1,2470934,81,0.0296582,80.8571
5,2020,45:50,Männlich,1,2648940,121,0.0360932,137.116
6,2020,50:55,Männlich,1,3383771,256,0.0420866,264.809
7,2020,55:60,Männlich,1,3370390,494,0.0386273,470.858
8,2020,60:65,Männlich,1,2779453,642,0.0319194,613.169
9,2020,65:70,Männlich,1,2312527,767,0.026885,741.597


## Reproduktion der Rohdaten-Visualisierung des Statistischen Bundesamts
Die [Sterbefallzahlen](https://www.destatis.de/DE/Themen/Gesellschaft-Umwelt/Bevoelkerung/Sterbefaelle-Lebenserwartung/sterbefallzahlen.html) zeigen eine Graphik der Sterbefälle pro Woche der Jahre 2016-2020.

Die Gestalt der Verläufe scheint übereinzustimmen.
- Die y-Achsen der Todesfälle liegt erheblich höher in unserer Analyse (**Tabellen-Check!!**)

In [416]:
# sum over alter, geschlecht

adjusted[:,:cell] = collect(zip(adjusted.jahr, adjusted.kw))
plot_data = combine(groupby(adjusted,:cell),
    :S => sum,
    :Sadj => sum,
    :Sadj2020 => sum)
plot_data[:,:Jahr] = [ x[1] for x in plot_data[:,1] ]
plot_data[:,:kw] = [ x[2] for x in plot_data[:,1] ];

In [417]:
plotly()
# Farben Bundesamt
lcolors = [ "#006298", "#A02438", "#449ADC", "#002B52", "#EC4A60" ]
yticks = 0:5000:30000
@df plot_data plot(:kw, :S_sum, group=:Jahr; lw=3, 
    palette=lcolors, 
    xlabel="Kalenderwoche",
    title="Wöchentliche Sterbefallzahlen in Deutschland",
    ylims=(0,32000), yticks=yticks, yformatter=:plain,
    legend=:bottomright
)

<!DOCTYPE html>
 
 
 Plots.jl

Die Alters- und Geschlechts-adjustierten Werte der Todesrate 
(korrigiert auf Alters- und Geschlechtsverteilung und bezogen auf Gesamtsterbezahl des Jahres 2020).
des Jahres 2020 sind bis zum Herbst unauffällig,
auch am Jahresende weniger auffällig als in den Rohdaten
(der Anteil älterer Bürger lag 2020 höher als im Schnitt).
Die Effekte 

In [418]:
@df plot_data plot(:kw, :Sadj2020_sum, group=:Jahr;lw=3,
    palette=lcolors, 
    xlabel="Kalenderwoche",
    title="Adjustierte Sterbefallzahlen in Deutschland",
    ylims=(0,32000), yticks=yticks, yformatter=:plain,
    legend=:bottomright
)

<!DOCTYPE html>
 
 
 Plots.jl

In [422]:
# select(srd,tuple(:S, :expected))
expected_deaths = combine(groupby(plot_data,:Jahr), 
    :S_sum => sum, 
    :Sadj_sum => sum, 
    :Sadj2020_sum => sum)
# Nicht korrigierte Rohdaten
@df expected_deaths plot(:Jahr,:S_sum_sum; label="data", lw=3,
    legend=:bottomright)
@df expected_deaths plot!(:Jahr,:Sadj_sum_sum; label="age-sex adjusted", lw=3)
@df expected_deaths plot!(:Jahr,:Sadj2020_sum_sum; label="age-sex adjusted 2020", lw=3)

<!DOCTYPE html>
 
 
 Plots.jl

In [402]:
plotattr(:Series)

Defined Series attributes are:
arrow, bar_edges, bar_position, bar_width, bins, colorbar_entry, connections, contour_labels, contours, extra_kwargs, fill_z, fillalpha, fillcolor, fillrange, group, hover, label, levels, line_z, linealpha, linecolor, linestyle, linewidth, marker_z, markeralpha, markercolor, markershape, markersize, markerstrokealpha, markerstrokecolor, markerstrokestyle, markerstrokewidth, normalize, orientation, primary, quiver, ribbon, series_annotations, seriesalpha, seriescolor, seriestype, show_empty_bins, smooth, stride, subplot, weights, x, xerror, y, yerror, z, zerror


# Anhang: Explorationen
Diese Berechnungen wurden verworfen aufgrund unklarer theoretische Bedeutung.

In [343]:
# Mittleres N
srd[:,:cell] = collect(zip(srd.alter,srd.geschlecht))
gsrd = groupby(srd, :cell)
N_mean = Dict([ r[1] => r[2] 
        for r in eachrow(combine(gsrd, :N=>mean))])
# N =sum(values(N_mean))
# P_mean = Dict( k => v /  for (k,v) in pairs(N_mean ) )
# sum(values(P_mean))

,alter,geschlecht,jahr,N,S,cell
,UnitRan…,String,Int64,Int64,Int64,Tuple…
1,0:30,Männlich,2016,12980804,5207,"(0:30, ""Männlich"")"
2,0:30,Männlich,2017,13049733,4827,"(0:30, ""Männlich"")"
3,0:30,Männlich,2018,13035105,4982,"(0:30, ""Männlich"")"
4,0:30,Männlich,2019,12994349,4820,"(0:30, ""Männlich"")"
5,0:30,Männlich,2020,12943028,4620,"(0:30, ""Männlich"")"
,alter,geschlecht,jahr,N,S,cell
,UnitRan…,String,Int64,Int64,Int64,Tuple…
1,85:200,Weiblich,2016,1536589,100313,"(85:200, ""Weiblich"")"
2,85:200,Weiblich,2017,1549439,100960,"(85:200, ""Weiblich"")"


In [272]:
adjusted = [
    ( jahr=j, alter=a, geschlecht=g,
      kw=kw,
    N = Bevölkerung(jahr=j,alter=a,geschlecht=g),
    S = S_(jahr=j, alter=a, geschlecht=g, kw=kw),
    marginalN = N_mean[ (a,g) ])
    for a in altersgruppen, 
        kw in 1:53, 
        g in geschlechter,
        j in jahre
    if S(jahr=j,alter=a,geschlecht=g,kw=kw) isa Int] |> DataFrame
            
# Formel 3
adjusted[:,:Sadj] = ( adjusted.S ./ adjusted.N ) .* adjusted.marginalN

adjusted[1:2,:]

,jahr,alter,geschlecht,kw,N,S,marginalN,Sadj
,Int64,UnitRan…,String,Int64,Int64,Int64,Float64,Float64
1,2016,0:30,Männlich,1,13511068,95,1.35614e7,95.354
2,2016,30:35,Männlich,1,3177640,37,3.2645e6,38.0114
